In [8]:
from google.colab import drive
drive.mount('/content/drive')

!pip install import-ipynb
import import_ipynb

%cd /content
%cd drive/MyDrive/VAE_Project_DGM
#%ll
#from ipynb.fs.full.VAE_Project.ipynb import *
from VAE_Project import Model

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
/content/drive/MyDrive/VAE_Project_DGM


In [9]:
import torch, torchvision
from torchvision import transforms
import numpy as np
#from VAE import Model
import matplotlib.pyplot as plt
import torch.nn as nn

from google.colab import drive
drive.mount('/content/drive')

def dequantization(x): #I have an issue with transforms.Lambda pickle, so I change to this function
    return x + torch.zeros_like(x).uniform_(0., 1./256.)

batch_size = 128
sample_size = 64
latent_dim = 100
lr = 1e-3

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

vae = Model(latent_dim=latent_dim,device=device).to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=lr)

checkpoint = torch.load("/content/drive/My Drive/VAE_Project_DGM/models/cifar10_batch128_mid100_epoch3650__VAE.pt",map_location=torch.device(device))
vae.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])


transform  = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(dequantization), #dequantization
    transforms.Normalize((0.,), (257./256.,)), #rescales to [0,1]
])
testset_cifar10 = torchvision.datasets.CIFAR10(root='./data/CIFAR10',train=False, download=True, transform=transform)
testloader_cifar10 = torch.utils.data.DataLoader(testset_cifar10,batch_size=1, shuffle=False, num_workers=2)

testset_svhn = torchvision.datasets.SVHN(root='./data/SVHN', download=True, transform=transform)
testloader_svhn = torch.utils.data.DataLoader(testset_svhn,batch_size=1, shuffle=False, num_workers=2)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files already downloaded and verified
Using downloaded and verified file: ./data/SVHN/train_32x32.mat


In [10]:
def test(vae, testloader, device, normalize=True):
    vae.eval()  # set to inference mode
    test_loss = []
    with torch.no_grad():
        # TODO
        #if ((epoch +1)%50 ==0):
        #    samples = vae.sample(64).to(device)
        #    torchvision.utils.save_image(samples,'/content/drive/My Drive/VAE_Project_DGM/samples/' + filename + 'epoch%d.png' % epoch)
        for inputs_rgb, _ in testloader:
            inputs = inputs_rgb
            inputs = inputs.to(device)
            output, mu, logvar = vae(inputs)
            if(normalize):
                test_loss.append(vae.loss(inputs, output, mu, logvar).item() / len(testloader.dataset))
            else:
                test_loss.append(vae.loss(inputs, output, mu, logvar).item())# / len(testloader.dataset))
    #test_loss /= len(testloader.dataset)
    return test_loss 

In [11]:
test_loss_per_batch_cifar10 = test(vae=vae, testloader=testloader_cifar10, device=device)
test_loss_per_batch_cifar10_unnormalized = test(vae=vae, testloader=testloader_cifar10, device=device, normalize=False)
print(test_loss_per_batch_cifar10)

with open('/content/drive/My Drive/VAE_Project_DGM/test_loss_per_batch_cifar10_VAE.txt', 'w') as writefile:
    writefile.write(str(test_loss_per_batch_cifar10))

[0.2029146240234375, 0.15368455810546874, 0.17781556396484374, 0.1845158203125, 0.20412945556640624, 0.1932606201171875, 0.16005277099609375, 0.1868498291015625, 0.19493380126953125, 0.183189990234375, 0.20977705078125, 0.18451416015625, 0.18475977783203126, 0.14304940185546874, 0.19576090087890624, 0.2018832763671875, 0.172291064453125, 0.198229296875, 0.13322314453125, 0.1897784912109375, 0.18733712158203125, 0.07733438720703124, 0.1874890869140625, 0.18666705322265625, 0.200769384765625, 0.1987882080078125, 0.184979345703125, 0.1861365966796875, 0.18863017578125, 0.178255615234375, 0.163593115234375, 0.205781591796875, 0.2097637451171875, 0.17714149169921875, 0.18005032958984374, 0.1742017333984375, 0.2047654052734375, 0.16735631103515625, 0.1955770263671875, 0.1976864990234375, 0.1960671142578125, 0.14771396484375, 0.17531328125, 0.18548341064453125, 0.17919061279296875, 0.18782984619140625, 0.18657974853515624, 0.2014177978515625, 0.20337369384765625, 0.19448836669921876, 0.168053

TypeError: ignored

In [ ]:
test_loss_per_batch_svhn = test(vae=vae, testloader=testloader_svhn, device=device)
test_loss_per_batch_svhn_unnormalized = test(vae=vae, testloader=testloader_svhn, device=device, normalize=False)
print(test_loss_per_batch_svhn)

with open('/content/drive/My Drive/VAE_Project_DGM/test_loss_per_batch_svhn_VAE.txt', 'w') as writefile:
    writefile.write(str(test_loss_per_batch_svhn))

[0.023665668855130738, 0.025258160123030053, 0.01978634174357911, 0.019429660545451286, 0.029273566380601853, 0.029269787144914478, 0.02779208432886533, 0.02775727470209758, 0.02643818647450073, 0.02544103813778035, 0.026915798215635535, 0.026907384917141026, 0.026206260045968304, 0.028525954244346955, 0.02712730210775506, 0.027018942355764637, 0.027044780463599042, 0.028411897311221113, 0.028919577971217768, 0.028719411786160366, 0.02722708026195449, 0.026728066182561905, 0.026966283005591615, 0.026892259642743183, 0.0256661976015304, 0.025052804987386018, 0.01986169483750273, 0.02088281832548852, 0.02897260391391778, 0.029035817796208896, 0.02500877555856437, 0.026839645283471204, 0.023589780869383643, 0.02452259055128691, 0.022926691639216217, 0.0231198575733377, 0.02102513121079726, 0.017094014573159393, 0.024288274606010348, 0.02231090284300903, 0.02380530894765176, 0.025162744419901854, 0.025004976326920978, 0.02694580048131066, 0.02878567838440354, 0.02831129432383595, 0.0278400